# Setup and imports

In [3]:
import warnings
warnings.simplefilter('ignore', FutureWarning)
import numpy as np
import scipy as scp
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (permutation_test_score, learning_curve, LeaveOneGroupOut,
                                     KFold, cross_val_score, cross_val_predict, cross_validate,
                                     train_test_split)
from sklearn.utils import parallel_backend
from sklearn.base import clone
from sklearn import datasets
from joblib.parallel import Parallel, delayed
import pickle
from permutation_helpers import random_data_gen, post_hoc_permutation
from simulate import simulate
from dask.distributed import progress, Client, wait

/home1/jrudoler/anaconda3/envs/py38/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Set up client

In [6]:
import cmldask.CMLDask as da
rhino_client = da.new_dask_client_slurm(
    job_name="simulations",
    memory_per_job="2GB",
    max_n_jobs=400, threads_per_job=1, 
    adapt=False,
    local_directory="/home1/jrudoler/dask-worker-space",
    log_directory="/home1/jrudoler/logs/",
#     resource_spec="h_vmem=2.5G,s_vmem=2.5G"
)

Unique port for jrudoler is 51360
{'dashboard_address': ':51360'}
To view the dashboard, run: 
`ssh -fN jrudoler@rhino2.psych.upenn.edu -L 8000:192.168.86.120:51360` in your local computer's terminal (NOT rhino) 
and then navigate to localhost:8000 in your browser
You've chosen to scale your cluster manually. This means workers will continue to run until you manually shut them down. Remember to run `client.shutdown` after you're done computing and no longer need to reserve resources.


In [4]:
#rhino_client.cluster.scale(400)

In [ ]:
rhino_client.shutdown()

# Post-hoc simulations

#### Post-hoc simulation parameters

In [4]:
### shared parameters
class_params = {
        "C":np.logspace(np.log10(1e-4), np.log10(1e5), 8),
        "class_weight":"balanced"
    }
permutation_params = {
        "n_permutations": 5000
    }
sim_params = {"n_sim": 500}
data_gen_params = {
    "maha":np.linspace(0., 1.5, 5)[4],
    "psi_diag": 1.0,
    "psi_offdiag": 0.,
    "ddof": 150
}
samplesize_params_post = {
    "sim":{
        "parameter_range":np.logspace(2, 5, 5).astype(int),
        **sim_params
    },
    "data_gen": {
        "maha":0.,
        "n_feats":10,
        "class_ratio":0.5,
        **data_gen_params
        
    },
    "classif": class_params,
    "perm": permutation_params
}

nfeats_params_post = {
    "sim":{
        "parameter_range":np.logspace(1, 10, 5, base=2).astype(int),
        **sim_params
    },
    "data_gen": {
        "n_samples":1000,
        "maha":0.,
        "class_ratio":0.5,
        **data_gen_params
    },
    "classif": class_params,
    "perm": permutation_params
}

ratio_params_post = {
    "sim":{
        "parameter_range": np.logspace(np.log10(.01), np.log10(.5), 5), #np.linspace(.1, .9, 5),
        **sim_params
    },
    "data_gen": {
        "n_samples":1000,
        "n_feats":10,
        "maha":0.,
        **data_gen_params
    },
    "classif": class_params,
    "perm": permutation_params
}

testsize_params_post = {
    "sim":{
        "parameter_range":np.logspace(np.log10(.01), np.log10(.5), 5),
        **sim_params
    },
    "data_gen": {
        "n_samples":1000,
        "n_feats":10,
        **data_gen_params
    },
    "classif": class_params,
    "perm": permutation_params
}

#### Simulation functions

In [5]:
@simulate(**samplesize_params_post["sim"])
def simulate_samplesize_post(param=None, seed=None, settings=samplesize_params_post):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## train model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    estimator.fit(X=X_train, y=y_train)
    y_pred = estimator.predict_proba(X_test)[:, 1]
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, permutation_scores, pvalue = post_hoc_permutation(
        y_true=y_test, y_score=y_pred,
        n_permutations=n_permutations, n_jobs=-1,
        )
    return score, permutation_scores, pvalue

@simulate(**testsize_params_post["sim"])
def simulate_testsize_post(param=None, seed=None, settings=testsize_params_post):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=param, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## train model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    estimator.fit(X=X_train, y=y_train)
    y_pred = estimator.predict_proba(X_test)[:, 1]
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, permutation_scores, pvalue = post_hoc_permutation(
        y_true=y_test, y_score=y_pred,
        n_permutations=n_permutations, n_jobs=-1,
        )
    return score, permutation_scores, pvalue

@simulate(**nfeats_params_post["sim"])
def simulate_nfeats_post(param=None, seed=None, settings=nfeats_params_post):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(n_feats=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(n_feats=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## train model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    estimator.fit(X=X_train, y=y_train)
    y_pred = estimator.predict_proba(X_test)[:, 1]
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, permutation_scores, pvalue = post_hoc_permutation(
        y_true=y_test, y_score=y_pred,
        n_permutations=n_permutations, n_jobs=-1,
        )
    return score, permutation_scores, pvalue

@simulate(**ratio_params_post["sim"])
def simulate_ratio_post(param=None, seed=None, settings=ratio_params_post):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(class_ratio=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(class_ratio=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## train model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    estimator.fit(X=X_train, y=y_train)
    y_pred = estimator.predict_proba(X_test)[:, 1]
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, permutation_scores, pvalue = post_hoc_permutation(
        y_true=y_test, y_score=y_pred,
        n_permutations=n_permutations, n_jobs=-1,
        )
    return score, permutation_scores, pvalue

Running 500 simulations
No dask client available, running sequentially
Running 500 simulations
No dask client available, running sequentially
Running 500 simulations
No dask client available, running sequentially
Running 500 simulations
No dask client available, running sequentially


#### Run functions

In [7]:
testsize_futures_post, testsize_gather = simulate_testsize_post()
samplesize_futures_post, samplesize_gather = simulate_samplesize_post()
nfeats_futures_post, nfeats_gather = simulate_nfeats_post()
ratio_futures_post, ratio_gather = simulate_ratio_post()

2500 parallel jobs
2500 parallel jobs
2500 parallel jobs
2500 parallel jobs


#### Gather results

In [7]:
import os
os.makedirs(f"sim_results/maha_{data_gen_params['maha']:.1f}", exist_ok=True)

In [8]:
wait(testsize_futures_post)
testsize_result = testsize_gather(testsize_futures_post) 
df_result = pd.DataFrame(testsize_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = testsize_params_post
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_testsize_post.pkl")
rhino_client.cancel(testsize_futures_post)

In [9]:
wait(samplesize_futures_post)
samplesize_result = samplesize_gather(samplesize_futures_post) 
df_result = pd.DataFrame(samplesize_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = samplesize_params_post
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_samplesize_post.pkl")
rhino_client.cancel(samplesize_futures_post)

In [10]:
wait(nfeats_futures_post)
nfeats_result = nfeats_gather(nfeats_futures_post)
df_result = pd.DataFrame(nfeats_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = nfeats_params_post
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_nfeats_post.pkl")
rhino_client.cancel(nfeats_futures_post)

In [11]:
wait(ratio_futures_post)
ratio_result = ratio_gather(ratio_futures_post)
df_result = pd.DataFrame(ratio_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = ratio_params_post
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_ratio_post.pkl")
rhino_client.cancel(ratio_futures_post)

In [23]:
rhino_client.cancel(samplesize_futures_post + ratio_futures_post + nfeats_futures_post + testsize_futures_post)

In [24]:
rhino_client.shutdown()

In [21]:
da.get_exceptions(ratio_futures_pre, np.arange(len(samplesize_futures_post)))

,exception,traceback_obj
param,,
35,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9abcecd00>
40,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9b89c7440>
105,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9de046f80>
120,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9dfab0300>
255,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9abc862c0>
256,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9bbbfdec0>
260,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9aab40f80>
295,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9df2aeb40>
330,ValueError('Only one class present in y_true. ...,<traceback object at 0x2ab9df71c4c0>


# Pre-training permutations (original)

In [10]:
def _train_score(estimator, X_train, X_test, y_train, y_test, 
                score_func, shuffle_labels=False):
    if shuffle_labels:
        indices = np.random.default_rng().permutation(len(y_train))
        y_train = y_train[indices]
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict_proba(X_test)[:,1]
    score = score_func(y_true=y_test, y_score=y_pred)
    return score



def pre_training_permutation(estimator, X_train, X_test, y_train, y_test,
                            n_permutations, score_func, verbose=False, n_jobs=None):
    score = _train_score(
        clone(estimator), X_train, X_test, y_train, y_test, score_func, shuffle_labels=False
    )
    permutation_scores = Parallel(n_jobs=n_jobs, verbose=verbose)(
        delayed(_train_score)(
            clone(estimator),
            X_train, X_test, y_train, y_test,
            score_func,
            shuffle_labels=True,
        )
        for _ in range(n_permutations)
    )
    permutation_scores = np.array(permutation_scores)
    pvalue = (np.sum(permutation_scores >= score) + 1.0) / (n_permutations + 1)
    return score, permutation_scores, pvalue

#### Simulation parameters

In [13]:
### shared parameters
# class_params = {
#     "C":1e-3,
#     "class_weight":"balanced"
# }
# permutation_params = {
#     "n_permutations": 5000
# }
# sim_params = {"n_sim": 150}
# data_gen_params = {
#     "maha":np.linspace(0., 1.5, 5)[0],
#     "psi_diag": 1.0,
#     "psi_offdiag": 0.,
#     "ddof": 150
# }
samplesize_params_pre = {
    "sim":{
        "parameter_range":np.logspace(2, 5, 5).astype(int),
        **sim_params
    },
    "data_gen": {
        "n_feats":10,
        "class_ratio":0.5,
        **data_gen_params
        
    },
    "classif": class_params,
    "perm": permutation_params
}

nfeats_params_pre = {
    "sim":{
        "parameter_range":np.logspace(1, 10, 5, base=2).astype(int),
        **sim_params
    },
    "data_gen": {
        "n_samples":1000,
        "class_ratio":0.5,
        **data_gen_params
    },
    "classif": class_params,
    "perm": permutation_params
}

ratio_params_pre = {
    "sim":{
        "parameter_range": np.logspace(np.log10(.01), np.log10(.5), 5), #np.linspace(.1, .9, 5),
        **sim_params
    },
    "data_gen": {
        "n_samples":1000,
        "n_feats":10,
        **data_gen_params
    },
    "classif": class_params,
    "perm": permutation_params
}

testsize_params_pre = {
    "sim":{
        "parameter_range":np.logspace(np.log10(.01), np.log10(.5), 5),
        **sim_params
    },
    "data_gen": {
        "n_samples":1000,
        "n_feats":10,
        "class_ratio":0.5,
        **data_gen_params
    },
    "classif": class_params,
    "perm": permutation_params
}

#### Simulation functions

In [14]:
@simulate(**samplesize_params_pre["sim"])
def simulate_samplesize_pre(param=None, seed=None, settings=samplesize_params_pre):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## set up model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, null, p = pre_training_permutation(
        estimator,
        X_train, X_test, y_train, y_test,
        n_permutations=n_permutations,
        score_func=roc_auc_score,
        verbose=True, n_jobs=-1
    )
    return score, null, p

@simulate(**testsize_params_pre["sim"])
def simulate_testsize_pre(param=None, seed=None, settings=testsize_params_pre):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=param, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## set up model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, null, p = pre_training_permutation(
        estimator,
        X_train, X_test, y_train, y_test,
        n_permutations=n_permutations,
        score_func=roc_auc_score,
        verbose=True, n_jobs=-1
    )
    return score, null, p

@simulate(**nfeats_params_pre["sim"])
def simulate_nfeats_pre(param=None, seed=None, settings=nfeats_params_pre):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(n_feats=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(n_feats=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## set up model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, null, p = pre_training_permutation(
        estimator,
        X_train, X_test, y_train, y_test,
        n_permutations=n_permutations,
        score_func=roc_auc_score,
        verbose=True, n_jobs=-1
    )
    return score, null, p

@simulate(**ratio_params_pre["sim"])
def simulate_ratio_pre(param=None, seed=None, settings=ratio_params_pre):
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(class_ratio=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(class_ratio=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## set up model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, null, p = pre_training_permutation(
        estimator,
        X_train, X_test, y_train, y_test,
        n_permutations=n_permutations,
        score_func=roc_auc_score,
        verbose=True, n_jobs=-1
    )
    return score, null, p

Running 500 simulations
Using dask client at http://192.168.86.101:51360/status
Running 500 simulations
Using dask client at http://192.168.86.101:51360/status
Running 500 simulations
Using dask client at http://192.168.86.101:51360/status
Running 500 simulations
Using dask client at http://192.168.86.101:51360/status


#### Run functions

In [15]:
testsize_futures_pre, testsize_gather = simulate_testsize_pre()
samplesize_futures_pre, samplesize_gather = simulate_samplesize_pre()
nfeats_futures_pre, nfeats_gather = simulate_nfeats_pre()
ratio_futures_pre, ratio_gather = simulate_ratio_pre()

2500 parallel jobs
2500 parallel jobs
2500 parallel jobs
2500 parallel jobs


#### Gather results

In [16]:
wait(testsize_futures_pre)
testsize_result = testsize_gather(testsize_futures_pre) 
df_result = pd.DataFrame(testsize_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = testsize_params_pre
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_testsize_pre.pkl")
rhino_client.cancel(testsize_futures_pre)

In [ ]:
wait(samplesize_futures_pre)
samplesize_result = samplesize_gather(samplesize_futures_pre) 
df_result = pd.DataFrame(samplesize_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = samplesize_params_pre
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_samplesize_pre.pkl")
rhino_client.cancel(samplesize_futures_pre)

In [ ]:
wait(nfeats_futures_pre)
nfeats_result = nfeats_gather(nfeats_futures_pre)
df_result = pd.DataFrame(nfeats_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = nfeats_params_pre
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_nfeats_pre.pkl")
rhino_client.cancel(nfeats_futures_pre)

In [ ]:
wait(ratio_futures_pre)
ratio_result = ratio_gather(ratio_futures_pre)
df_result = pd.DataFrame(ratio_result).melt(var_name="param")
df_result[["score", "perm_scores", "pval"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')
df_result._metadata = ratio_params_pre
df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_ratio_pre.pkl")
rhino_client.cancel(ratio_futures_pre)

In [35]:
rhino_client.cancel(ratio_futures_pre)

In [39]:
rhino_client.cancel(ratio_futures_pre+nfeats_futures_pre+samplesize_futures_pre+testsize_futures_pre)

In [35]:
da.get_exceptions(samplesize_futures_pre, range(len(samplesize_futures_pre)))

,exception,traceback_obj
param,,
0,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b42fe391ec0>
1,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b4320207f40>
2,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b432015bc00>
3,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b43099f7640>
4,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b43250bb100>
...,...,...
2495,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b4320f934c0>
2496,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b434257e440>
2497,"NameError(""name 'pre_training_permutation' is ...",<traceback object at 0x2b4342099880>


# Comparing runtime

In [1]:
import time
def runtime(f, *args, **kwargs):
    start = time.time()
    f(*args, **kwargs)
    stop = time.time()
    return stop - start

In [38]:
runtime_params = {
    "sim":{
        "parameter_range":np.logspace(2, 5, 5).astype(int),
        **sim_params
    },
    "data_gen": {
        "n_feats":10,
        "class_ratio":0.5,
        **data_gen_params
    },
    "classif": class_params,
    "perm": permutation_params
}

@simulate(**runtime_params["sim"])
def simulate_runtime_pre(param=None, seed=None, settings=runtime_params):
    start = time.time()
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## set up model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, null, p = pre_training_permutation(
        estimator,
        X_train, X_test, y_train, y_test,
        n_permutations=n_permutations,
        score_func=roc_auc_score,
        verbose=True, n_jobs=-1
    )
    stop = time.time()
    return stop - start

@simulate(**runtime_params["sim"])
def simulate_runtime_post(param=None, seed=None, settings=runtime_params):
    start = time.time()
    settings = deepcopy(settings)
    ## Simulate dataset
    X, y = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## Split into train-test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    ## Simulate validation set, same as original dataset
    X_val, y_val = random_data_gen(n_samples=param, seed=seed, **settings["data_gen"])
    ## iterate over possible penalty params
    max_AUC = 0
    best_C = None
    for C in settings["classif"].pop("C"):
        estimator = LogisticRegression(**settings["classif"], C=C)
        estimator.fit(X=X_train, y=y_train)
        y_pred = estimator.predict_proba(X_val)[:, 1]
        AUC = roc_auc_score(y_score=y_pred, y_true=y_val)
        best_C = C if AUC>max_AUC else best_C
    ## train model with tuned penalty
    estimator = LogisticRegression(**settings["classif"], C=best_C)
    estimator.fit(X=X_train, y=y_train)
    y_pred = estimator.predict_proba(X_test)[:, 1]
    ## permutations
    n_permutations = settings["perm"]["n_permutations"]
    score, permutation_scores, pvalue = post_hoc_permutation(
        y_true=y_test, y_score=y_pred,
        n_permutations=n_permutations, n_jobs=-1,
        )
    stop = time.time()
    return stop - start

Running 500 simulations
Using dask client at http://192.168.86.120:51360/status
Running 500 simulations
Using dask client at http://192.168.86.120:51360/status


In [39]:
runtime_futures_pre, runtime_gather = simulate_runtime_pre()
runtime_futures_post, runtime_gather = simulate_runtime_post()

2500 parallel jobs
2500 parallel jobs


In [49]:
# wait(runtime_futures_pre)

# df_result._metadata = runtime_params
# df_result.to_pickle(f"sim_results/maha_{data_gen_params['maha']:.1f}/simulate_ratio_pre.pkl")
rhino_client.cancel(runtime_futures_pre+runtime_futures_post)

In [47]:
runtime_result_pre = runtime_gather(runtime_futures_pre)
runtime_result_post = runtime_gather(runtime_futures_post)
df_result_pre = pd.DataFrame(runtime_result_pre).melt(var_name="param")
df_result_post = pd.DataFrame(runtime_result_post).melt(var_name="param")
df_result_pre['test'] = 'pre'
df_result_post['test'] = 'post'
df_result = pd.concat([df_result_pre, df_result_post])
df_result._metadata = runtime_params
df_result.to_pickle(f"sim_results/simulate_runtime.pkl")

# Hyperparameter tuning

In [5]:
import cmldask.CMLDask as da
rhino_client = da.new_dask_client_slurm(
    job_name="C_tuning",
    memory_per_job="1.5GB",
    max_n_jobs=50,
    threads_per_job=1, 
    adapt=True,
    local_directory="/home1/jrudoler/",
    log_directory="/home1/jrudoler/logs/",
)

Unique port for jrudoler is 51360
{'dashboard_address': ':51360'}
To view the dashboard, run: 
`ssh -fN jrudoler@rhino2.psych.upenn.edu -L 8000:192.168.86.106:51360` in your local computer's terminal (NOT rhino) 
and then navigate to localhost:8000 in your browser


In [16]:
# TODO: Return all AUC scores for all C values, not just the best one. 
@simulate(parameter_range=np.linspace(0., 1.5, 5), n_sim=200)
def test_best_C(param=None, seed=None):
    X, y = random_data_gen(n_samples=5000, n_feats=10, maha=param, class_ratio=0.5, seed=seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)
    estimator = LogisticRegressionCV(class_weight='balanced', Cs=np.logspace(np.log10(1e-3), np.log10(1e6), 15))
    estimator.fit(X_train, y_train)
    pred = estimator.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_true=y_test, y_score=pred), estimator.C_[0]

Running 200 simulations
Using dask client at http://192.168.86.101:51360/status


In [31]:
np.logspace(np.log10(1e-4), np.log10(1e6), 10)

array([1.00000000e-04, 1.29154967e-03, 1.66810054e-02, 2.15443469e-01,
       2.78255940e+00, 3.59381366e+01, 4.64158883e+02, 5.99484250e+03,
       7.74263683e+04, 1.00000000e+06])

In [19]:
C_futures, C_gather = test_best_C()

1000 parallel jobs


In [17]:
rhino_client.cancel(C_futures)

In [6]:
rhino_client.shutdown()

In [20]:
progress(C_futures)

VBox()

In [21]:
C_results = C_gather(C_futures)

In [22]:
df_result = pd.DataFrame(C_results).melt(var_name="param")
df_result[["score", "C"]] = df_result['value'].apply(pd.Series)
df_result = df_result.drop(columns='value')

In [23]:
pd.options.display.max_rows=None
df_result.groupby(["param", "C"]).describe()

score                                                    \
                    count      mean       std       min       25%       50%   
param C                                                                       
0.000 0.001000       51.0  0.500189  0.014283  0.472552  0.488991  0.500460   
      0.004394       40.0  0.500041  0.019120  0.462883  0.486804  0.498724   
      0.019307       27.0  0.501062  0.018689  0.476384  0.487279  0.494766   
      0.084834       25.0  0.502859  0.017194  0.468201  0.491137  0.500589   
      0.372759        8.0  0.495200  0.025852  0.453977  0.478291  0.498051   
      1.637894       18.0  0.502319  0.016872  0.476178  0.492074  0.500702   
      7.196857        7.0  0.503450  0.019665  0.480454  0.489900  0.502414   
      31.622777       6.0  0.488825  0.009346  0.476941  0.481053  0.490806   
      138.949549      5.0  0.483678  0.021437  0.455054  0.468840  0.491178   
      610.540230      6.0  0.498185  0.010658  0.483410  0.492416  0.496817   
      2682.695795     5.0  0.493460  0.007556  0.481519  0.493170  0.493292   
      51794.746792    1.0  0.498608       NaN  0.498608  0.498608  0.498608   
      227584.592607   1.0  0.545803       NaN  0.545803  0.545803  0.545803   
0.375 0.001000       15.0  0.594604  0.023669  0.551183  0.573382  0.596108   
      0.004394       15.0  0.599043  0.017942  0.571149  0.585270  0.599326   
      0.019307       20.0  0.602057  0.018132  0.571644  0.593126  0.603285   
      0.084834       18.0  0.600038  0.017353  0.578925  0.585954  0.596885   
      0.372759       33.0  0.602881  0.017407  0.578062  0.590328  0.597654   
      1.637894       32.0  0.603468  0.020501  0.562991  0.587174  0.604611   
      7.196857       22.0  0.601586  0.015666  0.580664  0.590070  0.597523   
      31.622777      20.0  0.599176  0.016564  0.555109  0.591145  0.600518   
      138.949549     18.0  0.603724  0.018639  0.564210  0.592799  0.602840   
      610.540230      1.0  0.611349       NaN  0.611349  0.611349  0.611349   
      2682.695795     4.0  0.592897  0.027000  0.570596  0.574472  0.585396   
      11787.686348    1.0  0.604891       NaN  0.604891  0.604891  0.604891   
      51794.746792    1.0  0.609516       NaN  0.609516  0.609516  0.609516   
0.750 0.001000        9.0  0.697900  0.020048  0.669878  0.679812  0.708005   
      0.004394        6.0  0.694352  0.010525  0.681569  0.686802  0.693118   
      0.019307       16.0  0.697783  0.012809  0.668708  0.693373  0.697207   
      0.084834       18.0  0.700481  0.012795  0.670866  0.692712  0.703244   
      0.372759       31.0  0.704001  0.016532  0.672575  0.690502  0.700693   
      1.637894       39.0  0.700479  0.017682  0.659323  0.691699  0.699717   
      7.196857       32.0  0.699809  0.020306  0.661055  0.687653  0.699470   
      31.622777      27.0  0.707750  0.016892  0.675732  0.696054  0.707129   
      138.949549     13.0  0.694726  0.021202  0.651374  0.686788  0.693359   
      610.540230      4.0  0.700462  0.011293  0.687362  0.694411  0.700163   
      2682.695795     5.0  0.700894  0.012433  0.678676  0.705791  0.706203   
1.125 0.001000        7.0  0.764602  0.012237  0.757236  0.758531  0.759809   
      0.004394        6.0  0.777013  0.013026  0.765176  0.767659  0.775545   
      0.019307        4.0  0.792030  0.004567  0.788618  0.789388  0.790402   
      0.084834       19.0  0.784655  0.018792  0.740827  0.781392  0.788342   
      0.372759       38.0  0.786210  0.014731  0.754517  0.777411  0.784494   
      1.637894       40.0  0.785741  0.014591  0.754877  0.776449  0.785682   
      7.196857       28.0  0.781825  0.014449  0.752816  0.774415  0.779885   
      31.622777      26.0  0.790781  0.010226  0.770363  0.785133  0.788923   
      138.949549     20.0  0.787554  0.018293  0.753575  0.775278  0.790003   
      610.540230      9.0  0.783217  0.017300  0.762592  0.775948  0.780088   
      11787.686348    1.0  0.790579       NaN  0.790579  0.790579  0.7